In [1]:
import pandas as pd

data = pd.read_csv('NSF_awardtopics.txt', error_bad_lines=False)
data.columns = ['NSF Award Title Non-phrases']
data['index'] = data.index
documents = data
print(data)

                            NSF Award Title Non-phrases  index
0     application applications basic premise data ba...      0
1     algorithm implementation algorithms analysis a...      1
2     assumption basic problems certain special case...      2
3     active updating amount analysis analytical res...      3
4     act act theory computer computer simulator com...      4
5     artificial intelligence basic query languages ...      5
6     abstract prosidic categories abstract prosodic...      6
7      dynamical  processes  energy -processing  inf...      7
8     ability analytically derived concepts approach...      8
9     acquisition ambiguous examples classification ...      9
10    advance applications artificial intelligence r...     10
11    addition arm assembly capabilities complex rob...     11
12    active experimentation additional knowledge ap...     12
13    addition areas cad/cam commercial reality comp...     13
14    ability addition analogical processing boilers...

b'Skipping line 946: expected 1 fields, saw 2\nSkipping line 1957: expected 1 fields, saw 2\nSkipping line 2826: expected 1 fields, saw 2\nSkipping line 4376: expected 1 fields, saw 2\nSkipping line 5810: expected 1 fields, saw 3\nSkipping line 5811: expected 1 fields, saw 3\n'


In [83]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
# from nltk.stem import WordNetLemmatizer, SnowballStemmer
# from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)

In [84]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /afs/crc.nd.edu/user/y/ywu6/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [85]:
stemmer = SnowballStemmer('english')
# def lemmatize_stemming(text):
#     return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    researchstopwords = ['data', 'research', 'model', 'student', 'idea', 'project']
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) >= 3 and token not in researchstopwords:
            result.append(token)
    return result

In [86]:
# doc_sample = documents[documents['index'] == 250].values[0][0]
# print('original document: ')
# words = []
# for word in doc_sample.split(' '):
#     words.append(word)
# print(words)
# print('\n\n tokenized and lemmatized document: ')
# print(preprocess(doc_sample))

In [87]:
processed_docs = documents['NSF Award Title Non-phrases'].map(preprocess)

In [88]:
processed_docs[:10]

0    [application, applications, basic, premise, ba...
1    [algorithm, implementation, algorithms, analys...
2    [assumption, basic, problems, certain, special...
3    [active, updating, analysis, analytical, resul...
4    [act, act, theory, simulator, teaching, system...
5    [artificial, intelligence, basic, query, langu...
6    [abstract, prosidic, categories, abstract, pro...
7    [dynamical, processes, energy, processing, inf...
8    [ability, analytically, derived, concepts, app...
9    [acquisition, ambiguous, examples, classificat...
Name: NSF Award Title Non-phrases, dtype: object

In [ ]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [ ]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [ ]:
# bow_doc_200 = bow_corpus[200]

# for i in range(len(bow_doc_200)):
#     print("Word {} (\"{}\") appears {} time.".format(bow_doc_200[i][0], 
#                                                      dictionary[bow_doc_200[i][0]], 
#                                                      bow_doc_200[i][1]))

In [ ]:
from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

In [ ]:
# from pprint import pprint

# for doc in corpus_tfidf:
#     pprint(doc)
#     break

In [ ]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=30, id2word=dictionary, passes=4, workers=6)

In [ ]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

In [ ]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=20, id2word=dictionary, passes=2, workers=4)

for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

In [ ]:
test_title = 'Computer Vision'
bow_vector = dictionary.doc2bow(preprocess(test_title))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

In [ ]:
test_title = 'Machine Learning'
bow_vector = dictionary.doc2bow(preprocess(test_title))

for index, score in sorted(lda_model_tfidf[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model_tfidf.print_topic(index, 5)))

In [ ]:
test_title = 'Models'
bow_vector = dictionary.doc2bow(preprocess(test_title))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))